* col, expr, cast, alias
* lit
* concat, concat_ws
* upper, lower, initcap

* `split`, `explode`
* `regexp_replace`, `regexp_extract`
* lpad, rpad
* ltrim, rtrim, trim

* type,

* `size` - array length
* `length` - string/list length


* current_date, current_timestamp, to_date, to_timestamp
* date_add, date_sub, datediff, months_between, add_months, next_day
* trunc, date_trunc
* year, month, weekeofyear, dayofyear, dayofmonth, dayofweek, hour minute second
* date_format
* unix_timestamp, from_unixtime

* coalesce, nvl 
* sql, selectExpr, expr

* na, fillna, replacena, dropna, isnan, isin
* isNull, isNotNull
* distinct, drop_duplicates, dropDuplicates
* df.na.drop, df.dropna

* sort, orderBy
* desc, asc, col -> asc(), asc_nulls_first(), asc_nulls_last(), desc(), desc_nulls_first(), desc_nulls_last()
* agg

* `spark.read.format(format).load()`
* `DataFrame.write.format(format).save()` (`DataFrame.write` -> `DataFrameWritter`)
* `option`, `options`, `schema`, `mode`
* `dataFrame.inputFiles()`


* `Column.astype()`, `Column.cast()`
* `pyspark.sql.functions.udf()` requires a retutn type unless return type is a string, `SparkSession.register.udf()`
  
`spark.udf.register` is used to register UDF to be invoked in Spark SQL query.
`pyspark.sql.functions.udf` is used to create UDF to be called when using DataFrame API.

- `join`, `union`, `broadcast`
-  `DataFrame.dropDuplicates`, `DataFrame.drop_duplicates` - alias  # subset as list only

## Actions - immediate execution
count, show, 

postponed execution until the action
select, 

## Parameters

`spark.conf.get()`, `spark.conf.set()`

* `spark.sql.repl.eagerEval.enabled` 
* `spark.sql.autoBroadcastJoinThreshold` - default 
* `spark.sql.shuffle.partitions` - default 200
* `spark.sql.parquet.compression.codec` - to overwrite default snappy compression


`cache`, `persist`, `unpersist`
* take, collect, parallelize

accumulator

Spark Execution Hierachy - **Job** -> **Shuffle** -> **Task**

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('instance').getOrCreate()
sc = spark.sparkContext

In [25]:
data = [('apple', 3), ('orange', 2), ('apple', 5), ('orange', 4)]
rdd = sc.parallelize(data)
result_rdd = rdd.groupByKey()


[('apple', <pyspark.resultiterable.ResultIterable object at 0x11f937890>), ('orange', <pyspark.resultiterable.ResultIterable object at 0x11abb1850>)]
<generator object <genexpr> at 0x104100520>


In [28]:
df = spark.createDataFrame([('apple', 3), ('orange', 2), ('apple', 5), ('orange', 4)], ['fruit', 'amount'])
df.printSchema()

root
 |-- fruit: string (nullable = true)
 |-- amount: long (nullable = true)



In [49]:
df.groupBy('fruit').sum().show()
df.groupBy(df.fruit).agg({'amount': 'sum'}).show()

from pyspark.sql.functions import sum
df.groupBy(df.fruit).agg(sum('amount')).show()

+------+-----------+
| fruit|sum(amount)|
+------+-----------+
| apple|          8|
|orange|          6|
+------+-----------+

+------+-----------+
| fruit|sum(amount)|
+------+-----------+
| apple|          8|
|orange|          6|
+------+-----------+

+------+-----------+
| fruit|sum(amount)|
+------+-----------+
| apple|          8|
|orange|          6|
+------+-----------+



In [64]:
from pyspark.sql import DataFrame
from pyspark.storagelevel import StorageLevel

#help(DataFrame.agg)

help(DataFrame.persist)
help(StorageLevel)

Help on function persist in module pyspark.sql.dataframe:

persist(self, storageLevel: pyspark.storagelevel.StorageLevel = StorageLevel(True, True, False, True, 1)) -> 'DataFrame'
    Sets the storage level to persist the contents of the :class:`DataFrame` across
    operations after the first time it is computed. This can only be used to assign
    a new storage level if the :class:`DataFrame` does not have a storage level set yet.
    If no storage level is specified defaults to (`MEMORY_AND_DISK_DESER`)
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Notes
    -----
    The default storage level has changed to `MEMORY_AND_DISK_DESER` to match Scala in 3.0.
    
    Parameters
    ----------
    storageLevel : :class:`StorageLevel`
        Storage level to set for persistence. Default is MEMORY_AND_DISK_DESER.
    
    Returns
    -------
    :class:`DataFrame`
        Persisted DataFrame.
    
    Examples
    --------
   

In [69]:
help(DataFrame.collect)

Help on function collect in module pyspark.sql.dataframe:

collect(self) -> List[pyspark.sql.types.Row]
    Returns all the records as a list of :class:`Row`.
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Returns
    -------
    list
        List of rows.
    
    Examples
    --------
    >>> df = spark.createDataFrame(
    ...     [(14, "Tom"), (23, "Alice"), (16, "Bob")], ["age", "name"])
    >>> df.collect()
    [Row(age=14, name='Tom'), Row(age=23, name='Alice'), Row(age=16, name='Bob')]



In [76]:
help(DataFrame.dropDuplicates) # - subset as list only
help(DataFrame.distinct) # - subset as list only
from pyspark.sql import Column
help(Column.isin)

Help on function dropDuplicates in module pyspark.sql.dataframe:

dropDuplicates(self, subset: Optional[List[str]] = None) -> 'DataFrame'
    Return a new :class:`DataFrame` with duplicate rows removed,
    optionally only considering certain columns.
    
    For a static batch :class:`DataFrame`, it just drops duplicate rows. For a streaming
    :class:`DataFrame`, it will keep all data across triggers as intermediate state to drop
    duplicates rows. You can use :func:`withWatermark` to limit how late the duplicate data can
    be and the system will accordingly limit the state. In addition, data older than
    watermark will be dropped to avoid any possibility of duplicates.
    
    :func:`drop_duplicates` is an alias for :func:`dropDuplicates`.
    
    .. versionadded:: 1.4.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    subset : List of column names, optional
        List of columns to use for duplicate comparison (de

In [77]:
df.drop?

Signature: df.drop(*cols: 'ColumnOrName') -> 'DataFrame'
Docstring:
Returns a new :class:`DataFrame` without specified columns.
This is a no-op if the schema doesn't contain the given column name(s).

.. versionadded:: 1.4.0

.. versionchanged:: 3.4.0
    Supports Spark Connect.

Parameters
----------
cols: str or :class:`Column`
    a name of the column, or the :class:`Column` to drop

Returns
-------
:class:`DataFrame`
    DataFrame without given columns.

Notes
-----
When an input is a column name, it is treated literally without further interpretation.
Otherwise, will try to match the equivalent expression.
So that dropping column by its name `drop(colName)` has different semantic with directly
dropping the column `drop(col(colName))`.

Examples
--------
>>> from pyspark.sql import Row
>>> from pyspark.sql.functions import col, lit
>>> df = spark.createDataFrame(
...     [(14, "Tom"), (23, "Alice"), (16, "Bob")], ["age", "name"])
>>> df2 = spark.createDataFrame([Row(height=80, name